In [48]:
# Imports
import pandas as pd
import matplotlib.pyplot as plt
import time 

import torch
import mrmr
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [49]:
# Constants
DATA_PATH = './data/'
DATASET_PATH = DATA_PATH + 'smartphone+based+recognition+of+human+activities+and+postural+transitions/'
MODELS_PATH = DATA_PATH + 'models/raw-models/'

In [50]:
# Load the data
train_set = pd.read_csv(DATA_PATH + "self-calculated/raw-data/train-data.txt", sep='\s+')
test_set = pd.read_csv(DATA_PATH + "self-calculated/raw-data/test-data.txt", sep='\s+')

train_set.columns=['accX', 'accY', 'accZ', 'gyroX', 'gyroY', 'gyroZ', "subject", "activity"]
test_set.columns=['accX', 'accY', 'accZ', 'gyroX', 'gyroY', 'gyroZ', "subject", "activity"]

In [51]:
# Creating model
device = None

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

class RawDataModel(nn.Module):
    def __init__(self):
        super(RawDataModel, self).__init__()
        self.num_layers = 2
        self.hidden_size = 16

        self.lstm = nn.LSTM(7, self.hidden_size, self.num_layers, batch_first=True)
        self.l1 = nn.Linear(self.hidden_size, 8)

    def forward(self, x):
        if len(x.shape) == 2:
            x = x.unsqueeze(0) 
        batch_size = x.size(0)

        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.l1(out[:, -1, :])

        if batch_size == 1:
            out = out.squeeze(0)

        return out
    
model = RawDataModel().to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [52]:
# Training and testing function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        y = y.squeeze()
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            y = y.squeeze()
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    return(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [53]:
# Dataset preparation
class HAPTDataset(Dataset):
    def __init__(self, dataset, features, label):
        self.data = torch.tensor(dataset[features].values, dtype=torch.float32)
        self.labels = torch.tensor(dataset[label].values, dtype=torch.float32)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]
    
train_dataset = HAPTDataset(train_set, train_set.columns[:-1], 'activity')
test_dataset = HAPTDataset(test_set, test_set.columns[:-1], 'activity')

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True)

In [54]:
epochs = 5
perf_timer = time.perf_counter()
perf_acc = ""

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_function, optimizer)
    perf_acc = test(test_dataloader, model, loss_function)
    
print(perf_acc)
perf_timer = time.perf_counter() - perf_timer
print("Done!")

Epoch 1
-------------------------------
loss: 61.318604  [    8/570908]
loss: 64.888290  [  808/570908]
loss: 92.366600  [ 1608/570908]
loss: 61.253162  [ 2408/570908]
loss: 70.748306  [ 3208/570908]
loss: 89.588898  [ 4008/570908]
loss: 54.575150  [ 4808/570908]
loss: 86.939819  [ 5608/570908]
loss: 44.195137  [ 6408/570908]
loss: 66.636627  [ 7208/570908]
loss: 66.368637  [ 8008/570908]
loss: 88.700851  [ 8808/570908]
loss: 41.451111  [ 9608/570908]
loss: 68.903122  [10408/570908]
loss: 72.621750  [11208/570908]
loss: 74.934059  [12008/570908]
loss: 74.834961  [12808/570908]
loss: 51.881752  [13608/570908]
loss: 68.437286  [14408/570908]
loss: 68.453934  [15208/570908]
loss: 64.389618  [16008/570908]
loss: 68.616974  [16808/570908]
loss: 41.543396  [17608/570908]
loss: 68.805038  [18408/570908]
loss: 87.349182  [19208/570908]
loss: 83.233475  [20008/570908]
loss: 85.187019  [20808/570908]
loss: 62.411819  [21608/570908]
loss: 70.628769  [22408/570908]
loss: 62.495502  [23208/570908]


RuntimeError: size mismatch (got input: [8], target: [4])